In [1]:
import pandas as pd
import numpy as np

In [2]:
# comment out unless google colab
from google.colab import files
uploaded = files.upload()

Saving Will_1k_set_1.csv to Will_1k_set_1.csv


In [3]:
# comment out unless google colab
import io
df = pd.read_csv(io.BytesIO(uploaded['Will_1k_set_1.csv']))

In [4]:
df['url'] = df['file_name'].apply(lambda x: 'https://commons.wikimedia.org/wiki/'+x.replace(" ","_"))

In [5]:
df1 = df[['url', 'depicts', 'parsed caption']]

In [6]:
import ast

df1['depicts'] = df1['depicts'].apply(ast.literal_eval)

<ipython-input-6-fd6255286db4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['depicts'] = df1['depicts'].apply(ast.literal_eval)


In [7]:
df1['count_depicts'] = df['depicts'].apply(lambda x: len(ast.literal_eval(x)))

<ipython-input-7-e82ea7bd64ed>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['count_depicts'] = df['depicts'].apply(lambda x: len(ast.literal_eval(x)))


In [8]:
df1['count_depicts'].value_counts()

count_depicts
1     624
2     166
3      86
4      64
5      35
6      11
8       6
7       5
10      2
9       1
Name: count, dtype: int64

In [9]:
df1['suggestions'] = None

<ipython-input-9-d99b59a87583>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['suggestions'] = None


In [10]:
# https://towardsdatascience.com/keyword-extraction-process-in-python-with-natural-language-processing-nlp-d769a9069d5c
# https://liaad.github.io/yake/docs/getting_started.html

In [10]:
!pip install yake

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 698.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.5 MB/s eta 0:00:00


In [11]:
import yake

In [12]:
for i in range(len(df1)):
  kw_extractor = yake.KeywordExtractor()
  text = str(df1["parsed caption"][i])
  language = "en"
  max_ngram_size = 3
  deduplication_threshold = 0.9
  numOfKeywords = 10
  custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
  keywords = custom_kw_extractor.extract_keywords(text)

  suggestions=[]
  for kw in keywords:
    suggestions.append(kw)
  df1['suggestions'][i] = suggestions

<ipython-input-12-a377bb25c9e6>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['suggestions'][i] = suggestions
<ipython-input-12-a377bb25c9e6>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['suggestions'][i] = suggestions
<ipython-input-12-a377bb25c9e6>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['suggestions'][i] = suggestions
<ipython-input-12-a377bb25c9e6>:14: SettingWithCopyWarning: 
A value is trying to be 

In [13]:
df1['suggestions'][0]

[('Wielkopolski w Poznaniu', 0.012602360123953448),
 ('Plac Wielkopolski', 0.023458380875189744),
 ('Poznaniu', 0.08596317751626563),
 ('Wielkopolski', 0.1447773057422032),
 ('Plac', 0.15831692877998726)]

In [14]:
df1['suggestion_tags'] = df1['suggestions'].apply(lambda x: [item[0] for item in x])

<ipython-input-14-2778d7505691>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['suggestion_tags'] = df1['suggestions'].apply(lambda x: [item[0] for item in x])


In [15]:
import requests
def get_qnums(tag_list):
  qnum_dict = {}
  for s in tag_list:
    try:
      url = f'https://www.wikidata.org/w/api.php?action=wbgetentities&sites=enwiki&titles={s}&languages=en&props=labels&format=json'
      response = requests.get(url).json()
      qnum = next(iter(response['entities']))
      if qnum.startswith('Q'):
        qnum_dict[s] = qnum
      else:
        #qnum_dict[s] = 'None'
        pass
    except:
      qnum_dict[s] = None
  return qnum_dict

In [16]:
df1['q_tags'] = df1['suggestion_tags'].apply(get_qnums)

<ipython-input-16-87cd6dbe8057>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['q_tags'] = df1['suggestion_tags'].apply(get_qnums)


In [17]:
df1['q_tags']

0       {'Wielkopolski': 'Q1460188', 'Plac': 'Q7200197'}
1      {'Horizontal bar': 'Q623270', 'Buenos Aires': ...
2      {'Siege of Lille': 'Q403479', 'France': 'Q142'...
3      {'Waycross Historic District': 'Q30622481', 'N...
4                                                     {}
                             ...                        
995    {'Sammy Arnold': 'Q21066283', 'Rugby player': ...
996    {'Náchod District': 'Q838308', 'Bild': 'Q15620...
997                    {'AIA': 'Q201734', 'Sun': 'Q525'}
998    {'Bild': 'Q156203', 'ADN': 'Q290466', 'Station...
999    {'Edit': 'Q410090', 'Gravel pit': 'Q504874', '...
Name: q_tags, Length: 1000, dtype: object

In [18]:
df1['num_q'] = df1['q_tags'].apply(lambda x: len(x))

<ipython-input-18-2d6bf9d89539>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['num_q'] = df1['q_tags'].apply(lambda x: len(x))


In [19]:
df1['num_q'].value_counts()

num_q
2    160
0    160
4    145
3    142
1    135
5    126
6     90
7     30
8      9
9      3
Name: count, dtype: int64

In [20]:
df1['suggestions_q'] = df1.apply(lambda row: [(key, value, row['q_tags'].get(key)) for key, value in row['suggestions'] if row['q_tags'].get(key) is not None], axis=1)

In [21]:
def bests(suggestions_q):
  return [(term, score, q) for term, score, q in suggestions_q if score < 0.05]

In [22]:
df1['best_suggest'] = df1['suggestions_q'].apply(bests)

In [23]:
df1

,url,depicts,parsed caption,count_depicts,suggestions,suggestion_tags,q_tags,num_q,suggestions_q,best_suggest
0,https://commons.wikimedia.org/wiki/File:Plac_W...,[Wielkopolski Square in Poznań],Plac Wielkopolski w Poznaniu.\n\n,1,"[(Wielkopolski w Poznaniu, 0.01260236012395344...","[Wielkopolski w Poznaniu, Plac Wielkopolski, P...","{'Wielkopolski': 'Q1460188', 'Plac': 'Q7200197'}",2,"[(Wielkopolski, 0.1447773057422032, Q1460188),...",[]
1,https://commons.wikimedia.org/wiki/File:2018-1...,[Yin Dehang],Horizontal bar qualification of the boys' arti...,1,"[(Horizontal bar qualification, 0.014632454545...","[Horizontal bar qualification, boys' artistic ...","{'Horizontal bar': 'Q623270', 'Buenos Aires': ...",2,"[(Horizontal bar, 0.048811108786045235, Q62327...","[(Horizontal bar, 0.048811108786045235, Q623270)]"
2,https://commons.wikimedia.org/wiki/File:CP_001...,[Siege of Lille],Lille (France - Nord department) - Destruction...,1,"[(Nord department, 0.007147515352778246), (rue...","[Nord department, rue de Tournai, Tournai due,...","{'Siege of Lille': 'Q403479', 'France': 'Q142'...",6,"[(Siege of Lille, 0.02727038941117262, Q403479...","[(Siege of Lille, 0.02727038941117262, Q403479..."
3,https://commons.wikimedia.org/wiki/File:Waycro...,[Waycross Historic District],"Waycross Historic District, Waycross, Georgia\...",1,"[(Waycross Historic District, 0.00094817452905...","[Waycross Historic District, Georgia House, Gi...","{'Waycross Historic District': 'Q30622481', 'N...",3,"[(Waycross Historic District, 0.00094817452905...","[(Waycross Historic District, 0.00094817452905..."
4,https://commons.wikimedia.org/wiki/File:J31_86...,[diesel generator],Zwei Dieselgeneratoren für die Gesamtversorgun...,1,"[(Gesamtversorgung des Artilleriewerkes, 0.001...","[Gesamtversorgung des Artilleriewerkes, Zwei D...",{},0,[],[]
...,...,...,...,...,...,...,...,...,...,...
995,https://commons.wikimedia.org/wiki/File:Sammy_...,[Sam Arnold],Sammy Arnold Irish Rugby player\n\n *[![](htt...,1,"[(Propagandakompanien der Wehrmacht, 0.0006536...","[Propagandakompanien der Wehrmacht, Arnold Iri...","{'Sammy Arnold': 'Q21066283', 'Rugby player': ...",2,"[(Sammy Arnold, 0.009683213106187576, Q2106628...","[(Sammy Arnold, 0.009683213106187576, Q2106628..."
996,https://commons.wikimedia.org/wiki/File:Houses...,[Končiny],"Houses 2 in Končiny, Zábrodí, Náchod District\...",1,"[(Propagandakompanien der Wehrmacht, 0.0006536...","[Propagandakompanien der Wehrmacht, politicall...","{'Náchod District': 'Q838308', 'Bild': 'Q15620...",3,"[(Náchod District, 0.003678542013331895, Q8383...","[(Náchod District, 0.003678542013331895, Q8383..."
997,https://commons.wikimedia.org/wiki/File:SDO_20...,[Sun],AIA 0304 channel is especially good at showing...,1,"[(cooler dense plumes, 0.003288255285582365), ...","[cooler dense plumes, filaments and prominence...","{'AIA': 'Q201734', 'Sun': 'Q525'}",2,"[(AIA, 0.04535371009006765, Q201734), (Sun, 0....","[(AIA, 0.04535371009006765, Q201734)]"
998,https://commons.wikimedia.org/wiki/File:Site_o...,[Kilbarchan railway station],Site of old Station\n\n *[![](https://upload....,1,"[(Propagandakompanien der Wehrmacht, 0.0006656...","[Propagandakompanien der Wehrmacht, politicall...","{'Bild': 'Q156203', 'ADN': 'Q290466', 'Station...",3,"[(Bild, 0.012347542149559911, Q156203), (ADN, ...","[(Bild, 0.012347542149559911, Q156203), (ADN, ..."


In [24]:
df1['url'][40]

'https://commons.wikimedia.org/wiki/File:Gliricidia_sepium_(Fabaceae)_01.jpg'

In [25]:
df1['suggestions'][40]

[('Gliricidia sepium flowers', 0.008321268142839145),
 ('Gliricidia sepium', 0.04940384002065631),
 ('sepium flowers', 0.04940384002065631),
 ('Gliricidia', 0.15831692877998726),
 ('flowers', 0.15831692877998726),
 ('sepium', 0.29736558256021506)]

In [26]:
df1['q_tags'][40]

{'Gliricidia sepium': 'Q2717303', 'Gliricidia': 'Q12549487'}

In [27]:
df1['suggestions_q'][40]

[('Gliricidia sepium', 0.04940384002065631, 'Q2717303'),
 ('Gliricidia', 0.15831692877998726, 'Q12549487')]

In [28]:
df1['best_suggest'][40]

[('Gliricidia sepium', 0.04940384002065631, 'Q2717303')]

In [29]:
df1['depicts'][40]

['Gliricidia sepium', 'flower']

In [32]:
df1['num_suggestions'] = df1['suggestions_q'].apply(lambda x: len(x))

In [34]:
df1['num_best'] = df1['best_suggest'].apply(lambda x: len(x))

In [39]:
df1['count_depicts'].value_counts()

count_depicts
1     624
2     166
3      86
4      64
5      35
6      11
8       6
7       5
10      2
9       1
Name: count, dtype: int64

In [41]:
df1['num_suggestions'].value_counts()

num_suggestions
2    160
0    160
4    145
3    142
1    135
5    126
6     90
7     30
8      9
9      3
Name: count, dtype: int64

In [42]:
df1['num_best'].value_counts()

num_best
0    433
1    280
2    162
3     59
4     34
5     22
6      7
7      3
Name: count, dtype: int64

In [47]:
def calc_percent(depicts, suggestions):
  if not depicts or not suggestions:
    return 0
  unique_depicts = set(depict.lower() for depict in depicts if isinstance(depict, str))
  matching = [item[0] for item in suggestions if item[0].lower() in unique_depicts]
  return (len(matching))/ len(depicts)

In [48]:
df1['dsp'] = df1.apply(lambda x: calc_percent(x['depicts'], x['suggestions_q']), axis=1)
df1['dspb'] = df1.apply(lambda x: calc_percent(x['depicts'], x['best_suggest']), axis=1)

In [50]:
df1['dsp'].value_counts()

dsp
0.000000    798
1.000000     97
0.500000     40
0.333333     26
0.250000     19
0.200000      7
0.166667      4
0.400000      2
0.125000      2
0.600000      1
0.666667      1
0.100000      1
0.285714      1
0.142857      1
Name: count, dtype: int64

In [51]:
df1['dspb'].value_counts()

dspb
0.000000    868
1.000000     80
0.500000     23
0.333333     14
0.250000      7
0.200000      3
0.125000      1
0.666667      1
0.100000      1
0.166667      1
0.142857      1
Name: count, dtype: int64

In [54]:
df_result = df1[['url', 'suggestions_q', 'best_suggest']]

In [55]:
df_result

,url,suggestions_q,best_suggest
0,https://commons.wikimedia.org/wiki/File:Plac_W...,"[(Wielkopolski, 0.1447773057422032, Q1460188),...",[]
1,https://commons.wikimedia.org/wiki/File:2018-1...,"[(Horizontal bar, 0.048811108786045235, Q62327...","[(Horizontal bar, 0.048811108786045235, Q623270)]"
2,https://commons.wikimedia.org/wiki/File:CP_001...,"[(Siege of Lille, 0.02727038941117262, Q403479...","[(Siege of Lille, 0.02727038941117262, Q403479..."
3,https://commons.wikimedia.org/wiki/File:Waycro...,"[(Waycross Historic District, 0.00094817452905...","[(Waycross Historic District, 0.00094817452905..."
4,https://commons.wikimedia.org/wiki/File:J31_86...,[],[]
...,...,...,...
995,https://commons.wikimedia.org/wiki/File:Sammy_...,"[(Sammy Arnold, 0.009683213106187576, Q2106628...","[(Sammy Arnold, 0.009683213106187576, Q2106628..."
996,https://commons.wikimedia.org/wiki/File:Houses...,"[(Náchod District, 0.003678542013331895, Q8383...","[(Náchod District, 0.003678542013331895, Q8383..."
997,https://commons.wikimedia.org/wiki/File:SDO_20...,"[(AIA, 0.04535371009006765, Q201734), (Sun, 0....","[(AIA, 0.04535371009006765, Q201734)]"
998,https://commons.wikimedia.org/wiki/File:Site_o...,"[(Bild, 0.012347542149559911, Q156203), (ADN, ...","[(Bild, 0.012347542149559911, Q156203), (ADN, ..."


In [56]:
df_result.to_csv('suggestions_1k.csv', index=False)

In [57]:
from google.colab import files

# Download the CSV file
files.download('suggestions_1k.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>